# Imports

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import glob

# Data

In [ ]:
df_2022 = pd.DataFrame()
paths = glob.glob('../data/fundos/inf_diario/*')
for path in paths:
    files = glob.glob(f'{path}/*.csv')
    for file in files:
        df_temp = pd.read_csv(file, encoding='ISO-8859-1', sep=';')
        df_2022 = pd.concat([df_2022, df_temp])
        print('Added file: ' + file)

Added file: ../data/fundos/inf_diario/inf_diario_fi_2020/inf_diario_fi_202002.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2020/inf_diario_fi_202010.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2020/inf_diario_fi_202012.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2020/inf_diario_fi_202009.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2020/inf_diario_fi_202003.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2020/inf_diario_fi_202004.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2020/inf_diario_fi_202006.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2020/inf_diario_fi_202007.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2020/inf_diario_fi_202011.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2020/inf_diario_fi_202005.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2020/inf_diario_fi_202008.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2020/inf_diario_fi_202001.csv
Added file: ../d

Added file: ../data/fundos/inf_diario/inf_diario_fi_2017/inf_diario_fi_201708.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2017/inf_diario_fi_201705.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2017/inf_diario_fi_201702.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2017/inf_diario_fi_201706.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2017/inf_diario_fi_201703.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2017/inf_diario_fi_201711.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2017/inf_diario_fi_201707.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2017/inf_diario_fi_201712.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2000_2004/inf_diario_fi_2000.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2000_2004/inf_diario_fi_2002.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2000_2004/inf_diario_fi_2004.csv
Added file: ../data/fundos/inf_diario/inf_diario_fi_2000_2004/inf_diario_fi_2001.csv
Adde